In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
from pathlib import Path

# ---------- 경로 설정 ----------
FOLDER = Path("/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종")

# ---------- CSV 불러오기 & 품목명 추출 ----------
all_items = []

for f in sorted(FOLDER.glob("*.csv")):
    try:
        df = pd.read_csv(f, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(f, encoding="cp949")
    
    # '품목명' 컬럼이 있으면 수집
    if "품목명" in df.columns:
        unique_items = df["품목명"].dropna().unique().tolist()
        for item in unique_items:
            all_items.append({"file": f.name, "품목명": item})
    else:
        print(f"[경고] '{f.name}' 파일에는 '품목명' 컬럼이 없습니다.")

# ---------- 결과 정리 ----------
if all_items:  # 하나라도 있으면 DataFrame 변환
    result_df = pd.DataFrame(all_items)

    # 파일별 품목명 리스트
    print("파일별 품목명:")
    print(result_df.groupby("file")["품목명"].apply(list))

    # 전체 unique 품목명
    print("\n전체 품목명 (중복 제거):")
    print(sorted(result_df["품목명"].unique()))

    # 저장
    result_df.to_csv(FOLDER / "품목명_모음.csv", index=False, encoding="utf-8-sig")
else:
    print("⚠️ 어떤 파일에서도 '품목명' 컬럼을 찾을 수 없습니다.")


파일별 품목명:
file
감자_cleaned.csv                                       [수미]
고구마_cleaned.csv                             [밤고구마, 호박고구마]
깐마늘_통합_최종_데이터.csv                                [깐마늘_통합]
깻잎_cleaned.csv                                   [깻잎(일반)]
느타리버섯_cleaned.csv         [느타리버섯(일반), 맛느타리버섯, 애느타리버섯(일반)]
단호박_cleaned.csv                                     [단호박]
당근_cleaned.csv                                   [당근(일반)]
미나리_cleaned.csv                           [돌미나리, 미나리(일반)]
부추_cleaned.csv                               [영양부추, 일반부추]
브로콜리_cleaned.csv                [뉴그린, 브로코리(국산), 브로코리(일반)]
새송이버섯_cleaned.csv                               [새송이(일반)]
시금치_cleaned.csv                                 [시금치(일반)]
애호박_cleaned.csv                              [애호박, 쥬키니호박]
양배추_cleaned.csv                        [양배추(수입), 양배추(일반)]
양상추_cleaned.csv                        [양상추(수입), 양상추(일반)]
양송이버섯_cleaned.csv                               [양송이(일반)]
양파_cleaned.csv                          [깐양파, 만생양파, 자주양파]


In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
from pathlib import Path
import re

# ===== 경로 설정 =====
FOLDER = Path("/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종")

# ===== 처리 규칙: {파일키워드: 드랍할 품목명} =====
rules = {
    "느타리버섯": "애느타리버섯(일반)",   # 1) 느타리버섯 CSV에서 드랍
    "브로콜리":   "브로콜리(일반)",       # 2) 브로콜리 CSV에서 드랍
    "오이":       "오이(일반)",           # 3) 오이 CSV에서 드랍
    "표고버섯":   "생표고(수입)"          # 4) 표고버섯 CSV에서 드랍
}

def find_csv_by_keyword(folder: Path, keyword: str) -> Path | None:
    """폴더 내에서 파일명에 keyword가 포함된 CSV를 우선순위 규칙으로 선택."""
    cand = [p for p in folder.glob("*.csv") if keyword in p.stem]
    if not cand:
        return None
    # 우선순위: 'cleaned' 포함 > 파일명 짧은 순
    cand.sort(key=lambda p: ( "cleaned" not in p.stem, len(p.name) ))
    return cand[0]

def read_csv_safely(path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path, encoding="utf-8-sig")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="cp949")

def sanitize_filename_piece(s: str) -> str:
    # 파일명에 쓰기 안전하도록 한글/영문/숫자/언더스코어/하이픈만 남김
    out = re.sub(r"[^\w\-가-힣]", "", s)
    return out[:50]  # 너무 길면 살짝 자르기

created = []

for keyword, drop_value in rules.items():
    src = find_csv_by_keyword(FOLDER, keyword)
    if src is None:
        print(f"⚠️ '{keyword}' 키워드로 CSV를 찾지 못했습니다.")
        continue

    df = read_csv_safely(src)

    if "품목명" not in df.columns:
        print(f"⚠️ {src.name}: '품목명' 컬럼이 없습니다. 스킵.")
        continue

    before = len(df)
    df2 = df[df["품목명"] != drop_value].copy()
    after = len(df2)
    dropped = before - after

    # 새 파일명: 원본파일명 + _drop_<품목명정리>.csv
    suffix = "_drop_" + sanitize_filename_piece(drop_value)
    dst = src.with_name(src.stem + suffix + ".csv")

    df2.to_csv(dst, index=False, encoding="utf-8-sig")
    created.append(dst.name)

    print(f"✅ {src.name} → {dst.name} 저장 (총 {before}행 → {after}행, 드랍 {dropped}행: '{drop_value}')")

if created:
    print("\n생성된 파일:")
    for name in created:
        print(" -", name)
else:
    print("\n새로 생성된 파일이 없습니다.")


✅ 느타리버섯_cleaned(애느타리).csv → 느타리버섯_cleaned(애느타리)_drop_애느타리버섯일반.csv 저장 (총 10626행 → 7084행, 드랍 3542행: '애느타리버섯(일반)')
✅ 브로콜리_(일반제외).csv → 브로콜리_(일반제외)_drop_브로콜리일반.csv 저장 (총 10626행 → 10626행, 드랍 0행: '브로콜리(일반)')
✅ 오이_cleaned.csv → 오이_cleaned_drop_오이일반.csv 저장 (총 10626행 → 7084행, 드랍 3542행: '오이(일반)')
✅ 표고버섯_cleaned.csv → 표고버섯_cleaned_drop_생표고수입.csv 저장 (총 7084행 → 7084행, 드랍 0행: '생표고(수입)')

생성된 파일:
 - 느타리버섯_cleaned(애느타리)_drop_애느타리버섯일반.csv
 - 브로콜리_(일반제외)_drop_브로콜리일반.csv
 - 오이_cleaned_drop_오이일반.csv
 - 표고버섯_cleaned_drop_생표고수입.csv


In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
from pathlib import Path
import unicodedata, re

# ===== 경로 =====
FOLDER = Path("/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종")

# ===== 유틸: 품목명 정규화 =====
def norm_name(s: str) -> str:
    if pd.isna(s):
        return ""
    # Unicode 정규화 + BOM/제로폭문자 제거
    s = unicodedata.normalize("NFKC", str(s)).replace("\ufeff", "").replace("\u200b","").replace("\u200c","")
    # 괄호 통일
    s = s.replace("（","(").replace("）",")").replace("[","(").replace("]",")").replace("{","(").replace("}",")")
    # 전부 소문자화 + 공백 제거
    s = re.sub(r"\s+", "", s.lower())
    return s

# ===== 처리 규칙 (브로콜리 + 표고버섯) =====
rules = [
    {
        "file_keywords": ["브로콜리", "브로코리"],   # 파일명 검색용
        "drop_values":  ["브로콜리(일반)", "브로코리(일반)"],  # 드랍할 품목 후보
        "suffix":       "drop_브로콜리일반"
    },
    {
        "file_keywords": ["표고버섯", "표고"],
        "drop_values":  ["생표고(수입)", "생 표고(수입)", "생표고(수입산)"],
        "suffix":       "drop_생표고수입"
    },
]

def find_csv(folder: Path, keywords: list[str]) -> Path | None:
    """폴더에서 파일명에 keyword 포함된 CSV 탐색"""
    cands = []
    for p in folder.glob("*.csv"):
        st = unicodedata.normalize("NFKC", p.stem)
        if any(k in st for k in keywords):
            cands.append(p)
    if not cands:
        return None
    # 'cleaned' 들어간 파일 우선, 그 다음 이름 짧은 순
    cands.sort(key=lambda p: ("cleaned" not in p.stem, len(p.stem)))
    return cands[0]

def read_csv(path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path, encoding="utf-8-sig")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="cp949")

created = []

for rule in rules:
    src = find_csv(FOLDER, rule["file_keywords"])
    if src is None:
        print(f"⚠️ 파일을 찾지 못했습니다: 키워드={rule['file_keywords']}")
        continue

    df = read_csv(src)
    if "품목명" not in df.columns:
        print(f"⚠️ {src.name}: '품목명' 컬럼이 없어 스킵합니다.")
        continue

    # 정규화 열 추가
    df["_품목명_norm"] = df["품목명"].map(norm_name)

    # 드랍 타겟 정규화 집합
    targets_norm = {norm_name(v) for v in rule["drop_values"]}

    # 마스크 계산
    drop_mask = df["_품목명_norm"].isin(targets_norm)
    before, drops = len(df), int(drop_mask.sum())
    df2 = df.loc[~drop_mask].drop(columns=["_품목명_norm"])
    after = len(df2)

    # 결과 저장
    dst = src.with_name(f"{src.stem}_{rule['suffix']}.csv")
    df2.to_csv(dst, index=False, encoding="utf-8-sig")

    created.append(dst.name)
    print(f"✅ {src.name} → {dst.name} 저장 | {before}→{after}행 (드랍 {drops}행)")
    if drops == 0:
        print("   ⚠️ 드랍된 행이 없습니다. 실제 '품목명' 값을 다시 확인해보세요.")

if created:
    print("\n생성된 파일 목록:")
    for n in created:
        print(" -", n)


✅ 브로콜리_(일반제외).csv → 브로콜리_(일반제외)_drop_브로콜리일반.csv 저장 | 10626→7084행 (드랍 3542행)
✅ 표고버섯_cleaned.csv → 표고버섯_cleaned_drop_생표고수입.csv 저장 | 7084→3542행 (드랍 3542행)

생성된 파일 목록:
 - 브로콜리_(일반제외)_drop_브로콜리일반.csv
 - 표고버섯_cleaned_drop_생표고수입.csv


In [6]:
# -*- coding: utf-8 -*-
import pandas as pd
from pathlib import Path

# ===== 경로 설정 =====
FOLDER = Path("/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종")
file_path = FOLDER / "대파.csv"

# ===== CSV 불러오기 =====
try:
    df = pd.read_csv(file_path, encoding="utf-8-sig")
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding="cp949")

# ===== 거래일자 컬럼을 datetime으로 변환 =====
if "거래일자" not in df.columns:
    raise KeyError("'거래일자' 컬럼이 없습니다. CSV 컬럼명을 확인하세요.")

# 문자열/숫자 → 날짜 변환
df["거래일자_dt"] = pd.to_datetime(df["거래일자"].astype(str), format="%Y%m%d", errors="coerce")

# ===== 필터링 (2015-10-19 이후만 남김) =====
cutoff = pd.to_datetime("2015-10-19")
before_rows = len(df)
df_filtered = df[df["거래일자_dt"] > cutoff].drop(columns=["거래일자_dt"])
after_rows = len(df_filtered)

# ===== 새 파일로 저장 =====
new_path = FOLDER / "대파_after_2015-10-19.csv"
df_filtered.to_csv(new_path, index=False, encoding="utf-8-sig")

print(f"✅ 저장 완료: {new_path}")
print(f"   (행 {before_rows} → {after_rows}, {before_rows - after_rows}행 드랍)")


✅ 저장 완료: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종/대파_after_2015-10-19.csv
   (행 10962 → 0, 10962행 드랍)


In [9]:
# -*- coding: utf-8 -*-
"""
대상: sojin/원본 전처리 최종/대파.csv
처리:
 1) '거래일자'가 2015-10-19 이하(≤)인 행 모두 제거 (NaT는 보존)
 2) '품목명'이 '대파(일반)' 인 행만 보존
결과: '원본 전처리 최종_after_2015-10-19/대파.csv'
"""

from pathlib import Path
import pandas as pd
import re

# -------- 경로 설정 --------
BASE_DIR = Path("/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin")
INPUT_DIR = BASE_DIR / "원본 전처리 최종"
INPUT_PATH = INPUT_DIR / "대파.csv"

OUTPUT_DIR = BASE_DIR / "원본 전처리 최종_after_2015-10-19"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH = OUTPUT_DIR / "대파.csv"
# ---------------------------

CUTOFF = pd.Timestamp("2015-10-19")

def read_csv_any(p: Path) -> pd.DataFrame:
    for enc in ("utf-8-sig", "cp949", "utf-8"):
        try:
            return pd.read_csv(p, encoding=enc)
        except Exception:
            continue
    raise RuntimeError(f"[읽기 실패] {p}")

def find_date_col(df: pd.DataFrame) -> str:
    canon = {re.sub(r"\s+", "", str(c)): c for c in df.columns}
    for key in ("거래일자", "일자", "날짜"):
        if key in canon:
            return canon[key]
    for c in df.columns:
        if re.sub(r"\s+", "", str(c)) == "거래일자":
            return c
    raise KeyError("이 파일에 '거래일자' 컬럼이 없습니다.")

def main():
    if not INPUT_PATH.exists():
        raise SystemExit(f"[에러] 대상 파일이 없습니다: {INPUT_PATH}")

    df = read_csv_any(INPUT_PATH)
    date_col = find_date_col(df)

    # 날짜 파싱
    dt = pd.to_datetime(df[date_col], errors="coerce")

    # 1) 날짜 조건: 2015-10-19 초과 or NaT
    mask_date = dt.isna() | (dt > CUTOFF)

    # 2) 품목명 조건: "대파(일반)" 만
    mask_item = df["품목명"] == "대파(일반)"

    # 최종 필터
    kept_df = df.loc[mask_date & mask_item].copy()

    # 저장
    kept_df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

    # 로그
    min_after = pd.to_datetime(kept_df[date_col], errors="coerce").min()
    print(f"[완료] {INPUT_PATH.name} → {OUTPUT_PATH}")
    print(f"원본 행수: {len(df)}, 보존 행수: {len(kept_df)}")
    print("보존된 최소 거래일자:", None if pd.isna(min_after) else str(min_after.date()))
    print("보존된 품목명 목록:", kept_df["품목명"].unique())

if __name__ == "__main__":
    main()


[완료] 대파.csv → /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종_after_2015-10-19/대파.csv
원본 행수: 10962, 보존 행수: 3542
보존된 최소 거래일자: 2015-10-20
보존된 품목명 목록: ['대파(일반)']


In [10]:
# -*- coding: utf-8 -*-
"""
대상: sojin/원본 전처리 최종/대파.csv
처리:
 1) '거래일자' <= 2015-10-19 행 제거 (NaT는 보존)
 2) '품목명' == '대파(일반)' 만 보존
결과: sojin/원본 전처리 최종_after_2015-10-19/대파_filtered.csv
"""

from pathlib import Path
import pandas as pd

# ---------------- 경로 설정 ----------------
BASE_DIR = Path("/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin")
INPUT_PATH = BASE_DIR / "원본 전처리 최종" / "대파.csv"

OUTPUT_DIR = BASE_DIR / "원본 전처리 최종_after_2015-10-19"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH = OUTPUT_DIR / "대파_filtered.csv"
# -------------------------------------------

CUTOFF = pd.Timestamp("2015-10-19")

# CSV 읽기 (인코딩 시도)
for enc in ("utf-8-sig", "cp949", "utf-8"):
    try:
        df = pd.read_csv(INPUT_PATH, encoding=enc)
        print(f"[INFO] {enc} 인코딩으로 읽기 성공")
        break
    except Exception as e:
        print(f"[WARN] {enc} 인코딩 실패: {e}")
else:
    raise SystemExit(f"[에러] {INPUT_PATH} 파일 읽기 실패")

# 거래일자 컬럼 확인
if "거래일자" not in df.columns:
    raise SystemExit("[에러] '거래일자' 컬럼이 없습니다")

# 날짜 파싱
df["거래일자"] = pd.to_datetime(df["거래일자"], errors="coerce")

# 조건 적용
mask_date = df["거래일자"].isna() | (df["거래일자"] > CUTOFF)
mask_item = df["품목명"] == "대파(일반)"

filtered_df = df.loc[mask_date & mask_item].copy()

# 결과 저장 (데이터가 비어 있어도 저장됨)
filtered_df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

# 로그 출력
print(f"[완료] {INPUT_PATH.name} → {OUTPUT_PATH}")
print(f"원본 행수: {len(df)}, 보존 행수: {len(filtered_df)}")
if not filtered_df.empty:
    print("보존된 최소 거래일자:", filtered_df["거래일자"].min())
    print("보존된 품목명 목록:", filtered_df["품목명"].unique())
else:
    print("[주의] 필터링 결과가 비어 있습니다!")


[WARN] utf-8-sig 인코딩 실패: 'utf-8' codec can't decode byte 0xb0 in position 1: invalid start byte
[INFO] cp949 인코딩으로 읽기 성공
[완료] 대파.csv → /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종_after_2015-10-19/대파_filtered.csv
원본 행수: 10962, 보존 행수: 3542
보존된 최소 거래일자: 2015-10-20 00:00:00
보존된 품목명 목록: ['대파(일반)']
